In [1]:
from glob import glob
from tqdm import tqdm
import pandas as pd
from utils import get_hour_crawl_df_lst

In [ ]:
last_week = sorted(glob('/mnt/sda/tnecniv/tnecniv-2023/*-*'))[-2]
last_week_hour_csv = sorted(glob(last_week+'/*-*.csv'))
result = {}
for csv in tqdm(last_week_hour_csv):
    df_lst = get_hour_crawl_df_lst(csv)
    for df in df_lst:
        res = df[['game_name', 'viewer_count']].groupby('game_name').sum().to_dict()['viewer_count']
        for k in res.keys():
            try:
                result[k]['vc'] += res[k]
                result[k]['ct'] += 1
            except:
                result[k] = {'vc': res[k], 'ct': 1}


In [12]:
r = sorted({k: [result[k]['vc'] / result[k]['ct'], result[k]['ct']] for k in result.keys()}.items(), key=lambda x:x[1][0], reverse=True)


In [26]:
pd.DataFrame([[s[0], s[1][1], s[1][0]] for s in r if s[0] != '']).merge(pd.read_csv('./result_collection/game_ratings.csv'), left_on=0, right_on='game_name', how='outer').drop('game_name', axis=1)

,0,1,2,ratings
0,Just Chatting,1008.0,315254.198413,F
1,Grand Theft Auto V,1008.0,192597.144841,T
2,League of Legends,1008.0,143060.480159,F
3,VALORANT,1008.0,119525.296627,F
4,Counter-Strike,1008.0,71668.023810,T
...,...,...,...,...
26721,Match Tree,NaN,NaN,F
26722,I'm on Merrymaking Watch,NaN,NaN,F
26723,Rune Classic,NaN,NaN,T
26724,Radar Rat Race,NaN,NaN,F


In [61]:
df = pd.DataFrame([[s[0], s[1][1], s[1][0]] for s in r if s[0] != '']).merge(pd.read_csv('./result_collection/game_ratings.csv'), left_on=0, right_on='game_name', how='outer').drop('game_name', axis=1)
df = df[(df[2] > 10000) & (df[1] > 1000)].dropna()
df[2] = df[2].astype(int)
df = df.drop(columns=1, axis=1)
df.columns = ['game_name', 'viewer count', 'ratings']
df

,game_name,viewer count,ratings
0,Just Chatting,315254,F
1,Grand Theft Auto V,192597,T
2,League of Legends,143060,F
3,VALORANT,119525,F
4,Counter-Strike,71668,T
5,Fortnite,58724,F
6,Dota 2,54335,F
7,Call of Duty: Warzone,42783,T
8,Apex Legends,41313,F
11,EA Sports FC 24,34508,F


In [56]:
fday = last_week_hour_csv[0].split('/')[-1][2:6]
lday = last_week_hour_csv[-1].split('/')[-1][2:6]
fday, lday

('0428', '0504')

In [60]:
df.to_csv(f'./result_collection/renew_game_ratings/{fday}_{lday}_simplified.csv', index=False)